In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/rk7VDaPjMp1h5VXS-cUyMg/league-of-legends-data-large.csv")
X = data.drop('win', axis=1)
y = data['win'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=1)
test_loader = DataLoader(test_dataset, batch_size=1)

In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)
    def forward(self, x):
        yhat = torch.sigmoid(self.linear(x))
        return yhat

In [ ]:
model = LogisticRegressionModel(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
epochs = 1000

for epoch in range(epochs):
    model.train()
    for x,y in train_loader:
        optimizer.zero_grad()
        yhat = model(x)
        loss = criterion(yhat, y.unsqueeze(0))
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
            print(f'Epoch: {epoch} Loss: {loss.item()}')

model.eval()
test_loss = 0.0
train_loss = 0.0
with torch.no_grad(): # Ensures no gradients are calculated during evaluation
    for x_train, y_train in train_loader:
        yhat = model(x_train)
        loss = criterion(yhat, y_train.unsqueeze(0))
        train_loss += loss.item()
    for x_test, y_test in test_loader:
        yhat = model(x_test)
        loss = criterion(yhat, y_test.unsqueeze(0))
        test_loss += loss.item()
        
yhat = model(X_train)
label = yhat > 0.5
print("The accuracy for train dataset: ", torch.mean((label == y_train.type(torch.ByteTensor)).type(torch.float)))
yhat = model(X_test)
label = yhat > 0.5
print("The accuracy for test dataset: ", torch.mean((label == y_test.type(torch.ByteTensor)).type(torch.float)))

In [ ]:
model_optimized = LogisticRegressionModel(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model_optimized.parameters(), lr=0.01, weight_decay=0.01)
epochs = 1000

for epoch in range(epochs):
    model.train()
    for x,y in train_loader:
        optimizer.zero_grad()
        yhat = model_optimized(x)
        loss = criterion(yhat, y.unsqueeze(0))
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
            print(f'Epoch: {epoch} Loss: {loss.item()}')

model_optimized.eval()
test_loss = 0.0
train_loss = 0.0
with torch.no_grad(): # Ensures no gradients are calculated during evaluation
    for x_train, y_train in train_loader:
        yhat = model_optimized(x_train)
        loss = criterion(yhat, y_train.unsqueeze(0))
        train_loss += loss.item()
    print(f'Train Loss: {train_loss}')
    for x_test, y_test in test_loader:
        yhat = model_optimized(x_test)
        loss = criterion(yhat, y_test.unsqueeze(0))
        test_loss += loss.item()
    print(f'Test Loss: {test_loss}')
    
y_predicted_train = model_optimized(X_train)
label_train = y_predicted_train > 0.5
print("The accuracy for train dataset: ", torch.mean((label == y_train.type(torch.ByteTensor)).type(torch.float)))
y_predicted_test = model_optimized(X_test)
label_test = y_predicted_test > 0.5
print("The accuracy for test dataset: ", torch.mean((label == y_test.type(torch.ByteTensor)).type(torch.float)))

The accuracy for train dataset:  tensor(0.5113)
The accuracy for test dataset:  tensor(0.5013)


In [ ]:
## Write your code here
# Save the model
torch.save(model_optimized.state_dict(), 'my_model_optimized.pth')
# Load the model
model2 = LogisticRegressionModel(X_train.shape[1])
model2.load_state_dict(torch.load('my_model_optimized.pth'))
# Ensure the loaded model is in evaluation mode
model2.eval()
# Evaluate the loaded model
test_loss = 0.0
train_loss = 0.0
with torch.no_grad(): # Ensures no gradients are calculated during evaluation
    for x_train, y_train in train_loader:
        yhat = model_optimized(x_train)
        loss = criterion(yhat, y_train.unsqueeze(0))
        train_loss += loss.item()
    print(f'Train Loss: {train_loss}')
    for x_test, y_test in test_loader:
        yhat = model_optimized(x_test)
        loss = criterion(yhat, y_test.unsqueeze(0))
        test_loss += loss.item()
    print(f'Test Loss: {test_loss}')
    
y_predicted_train = model_optimized(X_train)
label_train = y_predicted_train > 0.5
print("The accuracy for train dataset: ", torch.mean((label == y_train.type(torch.ByteTensor)).type(torch.float)))
y_predicted_test = model_optimized(X_test)
label_test = y_predicted_test > 0.5
print("The accuracy for test dataset: ", torch.mean((label == y_test.type(torch.ByteTensor)).type(torch.float)))


Epoch: 0 Loss: 0.9316130876541138
The accuracy for train dataset:  tensor(0.5113)
The accuracy for test dataset:  tensor(0.4750)


In [ ]:
epochs = 50
lrates = [0.01, 0.05, 0.1]

for lrate in lrates:
    model = LogisticRegressionModel(X_train.shape[1])
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lrate)
    for epoch in range(epochs):
        model.train()
        for x,y in train_loader:
            optimizer.zero_grad()
            yhat = model(x)
            loss = criterion(yhat, y.unsqueeze(0))
            loss.backward()
            optimizer.step()
        if epoch % 100 == 0:
                print(f'Epoch: {epoch} Loss: {loss.item()}')

    y_pred_test = model(X_test)
    label_test = y_pred_test > 0.5
    print(f"The accuracy for test dataset with lr={lrate}: ", torch.mean((label == y_test.type(torch.ByteTensor)).type(torch.float)))
